<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Data</a></span><ul class="toc-item"><li><span><a href="#Exploration-Data" data-toc-modified-id="Exploration-Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Exploration Data</a></span></li><li><span><a href="#Modeling-Data" data-toc-modified-id="Modeling-Data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Modeling Data</a></span></li></ul></li><li><span><a href="#Data-Exploration" data-toc-modified-id="Data-Exploration-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Exploration</a></span><ul class="toc-item"><li><span><a href="#Full-Data-Visualization" data-toc-modified-id="Full-Data-Visualization-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Full Data Visualization</a></span></li><li><span><a href="#Modeling-Data-Preparation" data-toc-modified-id="Modeling-Data-Preparation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Modeling Data Preparation</a></span></li></ul></li><li><span><a href="#Standard-Models" data-toc-modified-id="Standard-Models-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Standard Models</a></span><ul class="toc-item"><li><span><a href="#Two-Model" data-toc-modified-id="Two-Model-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Two Model</a></span></li><li><span><a href="#Interaction-Term" data-toc-modified-id="Interaction-Term-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Interaction Term</a></span></li><li><span><a href="#Class-Transformations" data-toc-modified-id="Class-Transformations-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Class Transformations</a></span><ul class="toc-item"><li><span><a href="#Binary-Transformation" data-toc-modified-id="Binary-Transformation-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Binary Transformation</a></span></li><li><span><a href="#Quaternary-Transformation" data-toc-modified-id="Quaternary-Transformation-3.3.2"><span class="toc-item-num">3.3.2&nbsp;&nbsp;</span>Quaternary Transformation</a></span></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Evaluation</a></span></li></ul></li><li><span><a href="#Generalized-Random-Forest" data-toc-modified-id="Generalized-Random-Forest-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Generalized Random Forest</a></span><ul class="toc-item"><li><span><a href="#Modeling" data-toc-modified-id="Modeling-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Modeling</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Evaluation</a></span></li></ul></li><li><span><a href="#Summary" data-toc-modified-id="Summary-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Summary</a></span></li></ul></div>

In [2]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from causeinfer.data import hillstrom
from causeinfer.utilities import plot_unit_distributions, train_test_split, over_sample
from causeinfer.standard_algorithms import TwoModel, InteractionTerm
from causeinfer.standard_algorithms import BinaryClassTransformation
from causeinfer.standard_algorithms import QuaternaryClassTransformation

pd.set_option("display.max_rows", 16)
pd.set_option('display.max_columns', None)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

ModuleNotFoundError: No module named 'causeinfer.data'

In [ ]:
os.getcwd()

In [ ]:
head_shape = helpers_py.head_shape

# Load Data

**Hillstrom Dataset**

An email marketing dataset from Kevin Hillstrom's MineThatData blog.

In [ ]:
hillstrom.download_hillstrom()

## Exploration Data

In [ ]:
# The full mostly unformatted dataset is loaded
data_raw = hillstrom.load_hillstrom(user_file_path="datasets/hillstrom.csv",
                                    format_covariates=False, 
                                    normalize=False)

df_full = pd.DataFrame(data_raw["dataset_full"], 
                       columns=data_raw["dataset_full_names"])

display(df_full.head())
df_full.shape

## Modeling Data

In [ ]:
# The formatted dataset is loaded
data_hillstrom = hillstrom.load_hillstrom(user_file_path="datasets/hillstrom.csv",
                                          format_covariates=True, 
                                          normalize=True)

df = pd.DataFrame(data_hillstrom["dataset_full"], 
                  columns=data_hillstrom["dataset_full_names"])

In [ ]:
# Covariates, treatments and responses are loaded separately
X = data_hillstrom["features"]

y = data_hillstrom["response_visit"] # response_visit, response_spend or response_conversion

# 1 is men's campaign, 2 is women's, and 0 is control
w = data_hillstrom["treatment"]

# Data Exploration

In [ ]:
sns.set(style="whitegrid")

## Full Data Visualization

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True, figsize=(20,5))

plot_unit_distributions(df=df_full, variable='variable', treatment = None,
                        plot_x_lab='x', plot_y_lab='y', plot_title='title', 
                        bins=None, fontsize=25, figsize=None, axis=ax1),

plot_unit_distributions(df=df_full, variable='variable', treatment = 'treatment',
                        plot_x_lab='x', plot_y_lab='y', plot_title='title', 
                        bins=None, fontsize=25, figsize=None, axis=ax2)

## Modeling Data Preparation

In [ ]:
df.describe()

In [ ]:
pd.crosstab(df['treatment'], df['visit'], margins = True, normalize=True)

In [ ]:
# Counts for treatment
control_indexes = [i for i, e in enumerate(w) if e == 0]
mens_indexes = [i for i, e in enumerate(w) if e == 1]
womens_indexes = [i for i, e in enumerate(w) if e == 2]

womens_mens_indexes = womens_indexes + mens_indexes

print(len(control_indexes))
print(len(mens_indexes))
print(len(womens_indexes))
print(len(womens_mens_indexes))

In [ ]:
X_control = X[control_indexes]
y_control = y[control_indexes]
w_control = w[control_indexes]

X_women = X[womens_indexes]
y_women = y[womens_indexes]
w_women = w[womens_indexes]

In [ ]:
# Over-sampling of control
X_os, y_os, w_os = over_sample(X_1=X_control, y_1=y_control, w_1=w_control, 
                               sample_2_size=len(X_women), shuffle=True)

In [ ]:
X_split = np.append(X_os, X_women, axis=0)
y_split = np.append(y_os, y_women, axis=0)
w_split = np.append(w_os, w_women, axis=0)

X_split.shape, y_split.shape, w_split.shape # should be equal in the first dimension

In [ ]:
X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(X_split, y_split, w_split, 
                                                                     percent_train=0.7, random_state=42, 
                                                                     maintain_proportions=True)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape, w_train.shape, w_test.shape

In [ ]:
print(np.array(np.unique(w_train, return_counts=True)).T)
print(np.array(np.unique(w_test, return_counts=True)).T)

# Standard Models

## Two Model

In [ ]:
two_model = TwoModel()

In [ ]:
two_model.fit(X_train, y_train, w_train)

In [ ]:
two_model_effects = two_model.predict(X_test)

## Interaction Term

In [ ]:
interaction_term = InteractionTerm()

In [ ]:
interaction_term.fit(X_train, y_train, w_train)

In [ ]:
interaction_term_effects = interaction_term.predict(X_test)

## Class Transformations

### Binary Transformation

In [ ]:
bct = BinaryClassTransformation()

In [ ]:
bct.fit(X_train, y_train, w_train)

In [ ]:
bct_effects = bct.predict(X_test)

### Quaternary Transformation

In [ ]:
qct = QuaternaryClassTransformation()

In [ ]:
qct.fit(X_train, y_train, w_train)

In [ ]:
qct_effects = qct.predict(X_test)

## Evaluation

# Generalized Random Forest

## Modeling

## Evaluation

# Summary